In [1]:
from tensorflow.python.keras.datasets import cifar10

/Users/yoshidayuusei/.pyenv/versions/3.6.5/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
(X_train, y_train) , (X_test, y_test) = cifar10.load_data()

In [3]:
print('X_train.shape :', X_train.shape)
print('y_train.shape :', y_train.shape)
print('X_test.shape :', X_test.shape)
print('y_test.shape :', y_test.shape)

X_train.shape : (50000, 32, 32, 3)
y_train.shape : (50000, 1)
X_test.shape : (10000, 32, 32, 3)
y_test.shape : (10000, 1)


In [4]:
from tensorflow.python.keras.utils import to_categorical

In [5]:
#特徴量の正則化
X_train = X_train/255
X_test = X_test/255

#クラスベクトルの1-hotベクトル化
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

print('X_train.shape :', X_train.shape)
print('y_train.shape :', y_train.shape)
print('X_test.shape :', X_test.shape)
print('y_test.shape :', y_test.shape)

X_train.shape : (50000, 32, 32, 3)
y_train.shape : (50000, 10)
X_test.shape : (10000, 32, 32, 3)
y_test.shape : (10000, 10)


In [6]:
from tensorflow.python.keras.models import Sequential

model = Sequential()

In [7]:
#畳み込み層ではConv2Dを用いる
from tensorflow.python.keras.layers import Conv2D

model.add(
    Conv2D(
        filters=32,#出力のチャンネル数
        input_shape=(32, 32, 3),
        kernel_size=(3, 3),#奇数*奇数の正方形であることが多い
        strides=(1, 1),#カーネルを動かす幅を指定する
        padding='same',
        #データの幅を扱う。ここではデータを畳み込んだときに入力と出力が等しくなるようにする。周りはゼロでおおう。
        #ゼロパディングしないとき：padding='valid'
        activation='relu'
    )
)

model.add(
    Conv2D(
        filters=32,#出力のチャンネル数
        kernel_size=(3, 3),
        strides=(1, 1),
        padding='same',
        activation='relu'
    )
)

In [8]:
#プーリング層の追加
#よく使われるマックスプーリングを使用
#ある領域の最大値を取ってきてサイズを縮小する
from tensorflow.python.keras.layers import MaxPooling2D

model.add(MaxPooling2D(pool_size=(2, 2)))

In [9]:
#ドロップアウトレーヤーの追加
#層の中のニューロンをランダムに無効化し、パラメーターの自由度を抑える
#=>過学習を防ぐ
#more ：　入力値からランダムにrate分だけ無効化し、強制的に０にする
#過度なdropoutは学習不足、学習速度の低下を招く
from tensorflow.python.keras.layers import Dropout

model.add(Dropout(0.25))

In [10]:
#畳み込み層の追加
model.add(
    Conv2D(
        filters=64,
        kernel_size=(3, 3),
        strides=(1, 1),
        padding='same',
        activation='relu'
    )
)

model.add(
    Conv2D(
        filters=64,
        kernel_size=(3, 3),
        strides=(1, 1),
        padding='same',
        activation='relu'
        )
)

In [11]:
#プーリング層の追加
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

In [12]:
model.output_shape
#データ数,　縦, 横, チャンネル数

(None, 8, 8, 64)

In [13]:
#全結合層では2次元のテンソルしか受け取れない
#=>二次元のテンソルに変換するFlattenレイヤーを使う
from tensorflow.python.keras.layers import Flatten

In [14]:
model.add(Flatten())

model.output_shape

(None, 4096)

In [15]:
#全結合層の追加
from tensorflow.python.keras.layers import Dense

model.add(Dense(units=512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=10, activation='softmax'))

In [ ]:
#モデルのコンパイル
from tensorflow.python.keras.callbacks import TensorBoard

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

tsb = TensorBoard(log_dir='./logs/')
history_model1 = model.fit(
    X_train,
    y_train,
    batch_size=32,
    epochs=20,
    validation_split=0.2,
    callbacks=[tsb]
)

Train on 40000 samples, validate on 10000 samples
Epoch 1/20
40000/40000 [==============================] - 189s 5ms/step - loss: 1.6092 - acc: 0.4115 - val_loss: 1.2213 - val_acc: 0.5626
Epoch 2/20
40000/40000 [==============================] - 2924s 73ms/step - loss: 1.1842 - acc: 0.5761 - val_loss: 0.9826 - val_acc: 0.6509
Epoch 3/20
37760/40000 [===========================>..] - ETA: 11s - loss: 1.0182 - acc: 0.6383